IMPORT AND SETTINGS

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
import matplotlib.pyplot as plt
import pandas as pd
import cv2 
import itertools
import numpy as np
from sklearn.model_selection import train_test_split
import random

try:
    tf.config.experimental.enable_op_determinism()
    print("✅ Op Determinism Abilitato!")
except AttributeError:
    print("⚠️ Attenzione: La tua versione di TF è troppo vecchia per enable_op_determinism.")

def reset_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
SEEDS = [555, 123]

I0000 00:00:1765541482.225978 3263105 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1765541482.257177 3263105 cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1765541482.842511 3263105 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2.21.0-dev20251210
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ Op Determinism Abilitato!


W0000 00:00:1765541483.551869 3263105 gpu_device.cc:2456] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0a. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


DATASET LOADING AND MODELLING

In [2]:
def load_dataset():
    folder='dataset/images'
    data=[]
    for filename in sorted(os.listdir(folder)):
        img_path=os.path.join(folder,filename)
        img=cv2.imread(img_path) #opencv save in bgr
        data.append({
            'image':img,
            'filename':filename
        })

    print(len(data),'images loaded')
    print('file name is: ',data[0]['filename'], 'shape of the image is:  ', data[0]['image'].shape )

    label=pd.read_csv('dataset/raw/bbx_annotations.csv')
    print(label.shape, label.iloc[0]['filename'])
    #images order is random, and for 1 image you can have more class

    print('we have', len(label['class'].unique()), 'different classes')

    #replace biggger img with half sized ones
    #cv2.imwrite('resize_image/last_img_pre_downsampling.jpg',data[-100]['image'])
    for i,item in enumerate(data):
        if "upper" in item["filename"].lower():
            data[i]['image']=cv2.resize(
                data[i]['image'],
                (data[i]['image'].shape[1]//2,data[i]['image'].shape[0]//2)
                ,interpolation=cv2.INTER_AREA
            )
    #cv2.imwrite('resize_image/last_img_post_downsampling.jpg',data[-100]['image'])
       
    return data, label

FROM THE PURE DATASET TO THE TRAIN AND TEST DATA AND LABEL

In [3]:
def dataset_modelling(dataset,annotation):   
    dataset_df = pd.DataFrame(dataset) 
    label_map={'goalpost':0,
               'ball':1,
               'robot':2,
               'goalspot':3,
               'centerspot':4}
    def get_vector(classes_found):
        vec=np.zeros(5,dtype=int)

        for c in classes_found:
            if c in label_map:
                vec[label_map[c]]=1
        return list(vec)
    
    grouped = annotation.groupby('filename')['class'].apply(list).reset_index()
    grouped['label']=grouped['class'].apply(get_vector)
    final_annotation=grouped[['filename','label']]

    final_dataset= pd.merge(dataset_df, final_annotation[['filename', 'label']], on='filename', how='inner')
    final_dataset.to_csv('csv/temp/final_dataset.csv')
    final_dataset=final_dataset.drop(columns=['filename'])
    df_train, df_test = train_test_split(final_dataset, test_size=0.2, random_state=42)
    x_train = np.array(df_train['image'].tolist()).astype('float32') /255.0
    y_train = np.array(df_train['label'].tolist()).astype('float32')
    
    x_test = np.array(df_test['image'].tolist()).astype('float32') / 255.0
    y_test = np.array(df_test['label'].tolist()).astype('float32')
    return x_train, y_train,x_test,y_test
    

DOUBLING THE DATA 

In [4]:
def augment_train_set(x_train,y_train,aug_type):
    rng = np.random.RandomState(42)    
    if aug_type=='flip':
        x_flipped = np.flip(x_train, axis=2)
        y_flipped = y_train
        x_train_aug = np.concatenate([x_train, x_flipped], axis=0)
        y_train_aug = np.concatenate([y_train, y_flipped], axis=0)
    elif aug_type=='noise':
        noise = rng.normal(loc=0.0, scale=0.05, size=x_train.shape)
        x_noisy = x_train + noise
        x_noisy = np.clip(x_noisy, 0., 1.)
        x_train_aug = np.concatenate([x_train, x_noisy], axis=0)
        y_noise = y_train
        y_train_aug = np.concatenate([y_train, y_noise], axis=0)
    elif aug_type=='both':
        x_flipped = np.flip(x_train, axis=2)
        y_flipped = y_train
        noise = rng.normal(loc=0.0, scale=0.05, size=x_train.shape)
        x_noisy = x_train + noise
        x_noisy = np.clip(x_noisy, 0., 1.)
        y_noise = y_train
        x_train_aug = np.concatenate([x_train,x_flipped, x_noisy], axis=0)
        y_train_aug = np.concatenate([y_train,y_flipped, y_noise], axis=0)

    #avoid to have all noisy data in validation--> shuffle
    indices = np.arange(x_train_aug.shape[0])
    rng.shuffle(indices)
    x_train_aug = x_train_aug[indices]
    y_train_aug = y_train_aug[indices]

    return x_train_aug, y_train_aug

CREATE ALL DIFFERENT COMBINATION OF HYPERPARAMETER, WITHOUT USING GRIDSEARCH

In [5]:
def create_hyperparam_combination():
    param_grid = {
    'batch_size': [16],
    'layer_number':[4,5],
    'kernel_dim': [7],
    'pool_dim': [3], 
    'lr': [0.0001,0.001],
    'fc1' : [128],
    'fc2': [128]     
}

    #every possible combination
    keys, values = zip(*param_grid.items())
    combinations = list(itertools.product(*values))
    combinations_dicts = [dict(zip(keys, v)) for v in combinations]
    return combinations_dicts

MODEL BUILIDNG, 
kernel dimesnsion, pooling dimension, fc layers dimension, number of conv layer and learning rate have different combination.
instead, i fixed:
pooling stride=2 
pooling type: avg pooling
number of kernel per layer: 16, 32, 64...
last pooling: glob avg pool

In [6]:
def build_model(layer_num,kernel_dim,pool_dim,fc1,fc2):
    model=models.Sequential()
    model.add(tf.keras.Input(shape=(240,320,3)))

    for i in range(layer_num):
        kernel_number=16*(2**i)
        model.add(layers.Conv2D(kernel_number,(kernel_dim,kernel_dim),activation='relu',padding='same'))
        model.add(layers.AveragePooling2D((pool_dim,pool_dim),strides=2,padding='same'))
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(fc1,activation='relu'))
    model.add(layers.Dense(fc2,activation='relu'))
    model.add(layers.Dense(5,activation='sigmoid'))

    #model.summary()
    return model


SAVING THE CSV FILE CONTAINING F1S OF EACH SEARCHED MODEL

In [7]:
def saving_csv(report_dict,param,target_names,all_results,seed,aug_type):
    current_result = param.copy()    
    for class_name in target_names:
        current_result[f'f1_{class_name}'] = report_dict[class_name]['f1-score']
    current_result['f1_macro_avg'] = report_dict['macro avg']['f1-score']

    all_results.append(current_result)
    pd.DataFrame(all_results).to_csv(f'csv/f1_search/search6{aug_type}_{seed}.csv', index=False)



MAIN

In [8]:
dataset, annotation=load_dataset()
x_train,y_train, x_test, y_test=dataset_modelling(dataset, annotation)
list_augtype=['flip','noise','both']
for aug_type in list_augtype:
    x_train_aug, y_train_aug = augment_train_set(x_train, y_train,aug_type)

    combination=create_hyperparam_combination()

    for seed in SEEDS:
        bestf1=0
        all_results = []

        print(f"\n ================== INIZIO CICLO CON SEED: {seed} ================== ")
        for i,param in enumerate(combination):

            
            print(f"\nTRAINING RUN {i+1}/{len(combination)}")
            tf.keras.backend.clear_session()
            reset_seeds(seed)
            model=build_model( param['layer_number'],param['kernel_dim'],param['pool_dim'],param['fc1'],param['fc2'])
            opt=tf.keras.optimizers.Adam(learning_rate=param['lr'])
            model.compile(optimizer=opt,
                        loss='binary_crossentropy',
                        metrics=[tf.keras.metrics.Precision(name='precision'),
                                tf.keras.metrics.Recall(name='recall'),
                                ])
            early_stop=EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
            )
            reset_seeds(seed)
            #class_weights_dict=compute_class_weight(y_train)
            history=model.fit(
                x_train_aug,y_train_aug,
                epochs=120,
                batch_size=param['batch_size'], 
                validation_split=0.2, 
                callbacks=[early_stop]
                )

            y_pred=model.predict(x_test)
            predictions_binary = (y_pred > 0.5).astype(int)
            target_names = ['goalpost','ball','robot','goalspot','centerspot']
            report_dict = classification_report(y_test, predictions_binary, target_names=target_names, output_dict=True)
            f1_macro = report_dict['macro avg']['f1-score']
            if f1_macro>bestf1:
                best_report_dict=report_dict
                best_param=param
                bestf1=f1_macro
            saving_csv(report_dict,param,target_names,all_results,seed,aug_type)


        df_report = pd.DataFrame(best_report_dict).transpose()
        df_report = df_report.round(2)
        df_report['support'] = df_report['support'].astype(int)
        csv_path = f'csv/report/report_bestcomb6{aug_type}_{seed}.csv'
        df_report.to_csv(csv_path)
        print(best_param)




2452 images loaded
file name is:  lower_100056_jpg.rf.ec9852c66b4eee4a185317210a378f16.jpg shape of the image is:   (240, 320, 3)
(8125, 8) upper_604302_jpg.rf.6215ee30a829ec658154eb4d067dfdf5.jpg
we have 5 different classes

 ================== INIZIO CICLO CON SEED: 555 ================== 

TRAINING RUN 1/4


W0000 00:00:1765541486.410533 3263105 gpu_device.cc:2456] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0a. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
I0000 00:00:1765541486.488124 3263105 gpu_device.cc:2040] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9228 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 5070, pci bus id: 0000:01:00.0, compute capability: 12.0a


Epoch 1/120


I0000 00:00:1765541488.824589 3263359 cuda_dnn.cc:461] Loaded cuDNN version 91600


176/176 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - loss: 0.5225 - precision: 0.7941 - recall: 0.4733 - val_loss: 0.4999 - val_precision: 0.7969 - val_recall: 0.4629
Epoch 2/120
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 0.4643 - precision: 0.8266 - recall: 0.4956 - val_loss: 0.4669 - val_precision: 0.8548 - val_recall: 0.4761
Epoch 3/120
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - loss: 0.4481 - precision: 0.8358 - recall: 0.5034 - val_loss: 0.4570 - val_precision: 0.8542 - val_recall: 0.4835
Epoch 4/120
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - loss: 0.4379 - precision: 0.8334 - recall: 0.5118 - val_loss: 0.4463 - val_precision: 0.8563 - val_recall: 0.4917
Epoch 5/120
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 0.4299 - precision: 0.8160 - recall: 0.5385 - val_loss: 0.4387 - val_precision: 0.8592 - val_recall: 0.4983
Epoch 6/120
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - loss: 0.4234 - precision: 0.8141 - recall: 0.5511 - val_loss: 0.4316 - val_precision: 0.8527 - val_recall

...

/home/simone/Desktop/ml_homework2/ml_hw2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 1/120
264/264 ━━━━━━━━━━━━━━━━━━━━ 13s 44ms/step - loss: 0.5163 - precision: 0.7895 - recall: 0.4774 - val_loss: 0.5125 - val_precision: 0.7992 - val_recall: 0.4653
Epoch 2/120
264/264 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - loss: 0.5080 - precision: 0.8020 - recall: 0.4755 - val_loss: 0.5165 - val_precision: 0.7976 - val_recall: 0.4691
Epoch 3/120
264/264 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - loss: 0.5093 - precision: 0.8013 - recall: 0.4749 - val_loss: 0.5187 - val_precision: 0.7992 - val_recall: 0.4653
Epoch 4/120
264/264 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - loss: 0.5091 - precision: 0.8016 - recall: 0.4748 - val_loss: 0.5187 - val_precision: 0.7992 - val_recall: 0.4653
Epoch 5/120
264/264 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - loss: 0.5089 - precision: 0.8016 - recall: 0.4748 - val_loss: 0.5189 - val_precision: 0.7992 - val_recall: 0.4653
Epoch 6/120
264/264 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - loss: 0.5087 - precision: 0.8016 - recall: 0.4748 - val_loss: 0.5191 - val_precision: 0.

E0000 00:00:1765551459.885576 3263105 node_def_util.cc:682] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
{'batch_size': 16, 'layer_number': 4, 'kernel_dim': 7, 'pool_dim': 3, 'lr': 0.001, 'fc1': 128, 'fc2': 128}


/home/simone/Desktop/ml_homework2/ml_hw2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
